# ZSC Comparsion of Different Encoders

In [ ]:
import sys
import os
sys.path.append('.')
from baselines.datasets import representations_from_df, labels_from_df 
from baselines.io import load_baseline_model

data_folder = "BIOSCAN_5M_DNA_experiments/data"
 
#"BarcodeBERT_soft_penalty/data"

for model_name in ["BarcodeBERT", "DNABERT-2", "DNABERT-S", "NT", "Hyena_DNA"]:
    if model_name == "BarcodeBERT":
        checkpoints = {"BarcodeBERT-5M":"/h/pmillana/projects/BIOSCAN_5M_DNA_experiments/model_checkpoints/8_4_4/checkpoint_pretraining.pt",
                       #"BarcodeBERT-1.5M":"/scratch/ssd004/scratch/pmillana/checkpoints/canada-1.5M/k4-4-4_w1.0_m1.0_r0.0.pt",
                      }
        
        for ckpt_name in checkpoints:
            checkpoint = checkpoints[ckpt_name]
            embedder = load_baseline_model(model_name, checkpoint_path=checkpoint, new_vocab=False)

            print(ckpt_name)

            embedder.name = model_name
            
            # Ensure model is in eval mode
            embedder.model.eval()
        
            trainable_params = sum(	p.numel() for p in embedder.model.parameters() if p.requires_grad)
        
            print(f"Number of trainable parameters: {trainable_params}")
            
            for file in ["unseen", "supervised_test"]: 
                filename = f"/h/pmillana/projects/{data_folder}/{file}.csv"
                embeddings = representations_from_df(filename, embedder, dataset="BIOSCAN-5M", target="processid") #dataset= "BIOSCAN-5M"
                print(embeddings.shape)

            #os.rename(f"/scratch/ssd004/scratch/pmillana/embeddings/embeddings/BIOSCAN-5M/BarcodeBERT", 
            #          f"/scratch/ssd004/scratch/pmillana/embeddings/embeddings/BIOSCAN-5M/{ckpt_name}")
            
    else:
        embedder = load_baseline_model(model_name)
    
        embedder.name = model_name
        
        # Ensure model is in eval mode
        embedder.model.eval()
    
        trainable_params = sum(	p.numel() for p in embedder.model.parameters() if p.requires_grad)
    
        print(f"Number of trainable parameters: {trainable_params}")
        
        for file in ["unseen", "supervised_test"]: 
            filename = f"/h/pmillana/projects/{data_folder}/{file}.csv"
            embeddings = representations_from_df(filename, embedder, dataset="BIOSCAN-5M", target="processid") #dataset= "BIOSCAN-5M"
            print(embeddings.shape)

Using device: cuda

Loading model from /h/pmillana/projects/BIOSCAN_5M_DNA_experiments/model_checkpoints/8_4_4/checkpoint_pretraining.pt

Loading model from /h/pmillana/projects/BIOSCAN_5M_DNA_experiments/model_checkpoints/8_4_4/checkpoint_pretraining.pt
Loaded model from /h/pmillana/projects/BIOSCAN_5M_DNA_experiments/model_checkpoints/8_4_4/checkpoint_pretraining.pt
BarcodeBERT-5M
Number of trainable parameters: 129478144
Using device: cuda
Calculating embeddings for BarcodeBERT
/scratch/ssd004/scratch/pmillana/embeddings/embeddings/BIOSCAN-5M/BarcodeBERT/unseen.pickle


0it [00:00, ?it/s]

(3396, 768)
Using device: cuda
Calculating embeddings for BarcodeBERT
/scratch/ssd004/scratch/pmillana/embeddings/embeddings/BIOSCAN-5M/BarcodeBERT/supervised_test.pickle


0it [00:00, ?it/s]

(18348, 768)


...

/h/pmillana/miniconda3/envs/hyenadna/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/fs01/home/pmillana/projects/BarcodeBERT/baselines/models/dnabert2.py:203: UserWarning: Unable to import Triton; defaulting MosaicBERT attention                 implementation to pytorch (this will reduce throughput when using this model).
  self.self = BertUnpadSelfAttention(config)


Number of trainable parameters: 117068544
Using device: cuda
Calculating embeddings for DNABERT-S
/scratch/ssd004/scratch/pmillana/embeddings/embeddings/BIOSCAN-5M/DNABERT-S/unseen.pickle


0it [00:00, ?it/s]

(3396, 768)
Using device: cuda
Calculating embeddings for DNABERT-S
/scratch/ssd004/scratch/pmillana/embeddings/embeddings/BIOSCAN-5M/DNABERT-S/supervised_test.pickle


0it [00:00, ?it/s]

### This notebook Compares the embedding performance on the 'BIOSCAN-5M' dataset, of seven different DNA barcode-based models: BarcodeBERT-1M, BarcodeBERT-5M, DNABERT, DNABERT-2, DNABERT-S, Hyena_DNA and NT.

In [ ]:
%matplotlib inline
import os
import pickle
import cProfile
import pstats
import warnings

import numpy as np
import sklearn
from sklearn.neighbors import KNeighborsClassifier, KDTree
from scipy.spatial import distance
from matplotlib import pyplot as plt
from obj_knn import FinBOL_GBOL
from tqdm import tqdm

warnings.filterwarnings("ignore")

dataset = "BIOSCAN-5M"
data_folder = "BIOSCAN_5M_DNA_experiments/data"

### Each sample is labeled at seven taxonomic ranks: class, order, family, subfamily, tribe, genus, and species. 

In [ ]:
rank_list = ["class", "order", "family", "genus", "species", "dna_bin"]
#rank_list = ["dna_bin"]
#rank_list = ["species"]
encoders = ["BarcodeBERT", "DNABERT-2", "DNABERT-S", "Hyena_DNA", "NT"] #, "DNABERT"]

# Creating the embeddings dictionary to hold all the FinBOL vs GBOL embedding/label data 
embeddings = {}
for encoder in encoders:
    embeddings[encoder] = {}

### We load embeddings from the test and the unseen_test partitions

In [ ]:
for encoder in encoders:
    # reading the Seen embeddings
    embeddings_folder = f"/scratch/ssd004/scratch/pmillana/embeddings/embeddings/{dataset}"
    with open(f"{embeddings_folder}/{encoder}/supervised_test.pickle", "rb") as train_handle:
        embeddings[encoder]["Seen"] = pickle.load(train_handle)

    # reading the Unseen embeddings
    with open(f"{embeddings_folder}/{encoder}/unseen.pickle", "rb") as test_handle:
        embeddings[encoder]["Unseen"] = pickle.load(test_handle)

print('Partition lengths:')
print("Train: ", len(embeddings[encoder]["Seen"]['ids']))
print("Test: ", len(embeddings[encoder]["Unseen"]['ids']))


In [ ]:
for encoder in encoders:
    print(encoder)
    for x in (embeddings["BarcodeBERT"]["Unseen"]['ids'][0:3]):
        print(x)
    print('')


### Each model encodes a different represenation of the data it was given. To see this, and check that all the data from each model was downloaded properly, a small slice of each encoder's first three embeddings is shown below.

In [ ]:
# checking embeddings have been loaded correctly for each model
for encoder in encoders:
    print(encoder)
    for x in (embeddings[encoder]["Seen"]['data'][0:3]):
        print(x[0:5])
    print('')

### After reading the embedding files from each model, the labels associated with each embeddings must be further processed in order to be used in the ZSC. A single sample from the file contains a 'data' segment (the embedding), and a 'ids' segments which contains a label that holds several different kinds of information about that specific sample. It is this 'ids' segment that must be seperated into several more specific segments including the label of that sample at each of the seven taxonomic ranks listed above.

In [ ]:
import pandas as pd

train = pd.read_csv(f"/h/pmillana/projects/{data_folder}/supervised_test.csv")
test = pd.read_csv(f"/h/pmillana/projects/{data_folder}/unseen.csv")

df_dict = {'Seen': train, 'Unseen':test}

for encoder in encoders:
    for part in ('Seen', 'Unseen'):
        df = df_dict[part]
        for rank in rank_list:
            if rank not in ['class', 'dna_bin']:
                taxa = rank + '_name'
            else:
                taxa = rank
            processid_to_taxa = dict(zip(df['processid'], df[taxa]))
            # extract label at each taxonomic rank
            embeddings[encoder][part][rank] = [processid_to_taxa.get(processid, None) for processid in embeddings[encoder][part]['ids']]

### After processing the initial label, each sample now has 10 distinct labels that can be used to group and identify them. Below is an example of the labels concerning the first sample of the FinBOL dataset

In [ ]:
# displaying all the new labels
for x in embeddings['NT']['Seen'].keys():
    if x != 'data':
        print(f"{x}:{' '*(10-len(x))}" ,embeddings['Hyena_DNA']['Seen'][x][13])

In [ ]:
df_dict['Seen'][df_dict['Seen']["processid"]=="CRPEB17386-21"]

### A single dictionary called 'embeddings' now holds all the data and labels associated with each sample, for each partition, for each model. This example shows the general tree structure for how the data is accessed for each model where each key holds another dictionary except for those at the lowest level which are lists.

In [ ]:
print(embeddings.keys())
print(embeddings['NT'].keys())
print(embeddings['NT']['Unseen'].keys())

# ZSC Analysis
the results following the section as well as the associated output files for each model have been made using k=10 for the KNN analysis

In [ ]:
import umap
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import adjusted_mutual_info_score

def zsc_pipeline(X, y_true):
        
    # Step 1: Dimensionality reduction with UMAP to 50 dimensions
    umap_reducer = umap.UMAP(n_components=50, random_state=42)
    X_reduced = umap_reducer.fit_transform(X)
    
    # Step 2: Cluster the reduced embeddings with Agglomerative Clustering (L2, Ward’s method)
    agglomerative_clustering = AgglomerativeClustering(n_clusters=4363, linkage='ward')
    cluster_labels = agglomerative_clustering.fit_predict(X_reduced)
    
    # Step 3: Evaluate clustering performance with Adjusted Mutual Information (AMI) score
    ami_score = adjusted_mutual_info_score(y_true, cluster_labels)
    
    print("Adjusted Mutual Information (AMI) score:", ami_score) 
    return ami_score

In [ ]:
# dictionaries for storing results and knn probability data
results = {}
label_probs = {}

for encoder in encoders:
    #results[encoder] = {}

    # using tqdm to display progress bar for each model
    for i in tqdm(range(len(rank_list)),desc=f"{encoder}{' '*(15-len(encoder))}"):
        rank=rank_list[i]
        results[encoder][rank] = {}

        # creating a number mapping for the labels at the current taxonomic rank to use for the analysis
        #all_labels = sorted(list(set(embeddings[encoder]['Unseen'][rank]+embeddings[encoder]['Seen'][rank])))
        #embeddings[encoder]['Seen']['mapped_y_true'] = np.array([all_labels.index(el) for el in embeddings[encoder]['Seen'][rank]])
        #embeddings[encoder]['Unseen']['mapped_y_true'] = np.array([all_labels.index(el) for el in embeddings[encoder]['Unseen'][rank]])
        
        partition_name = "test_seen + unseen"
        X_part = np.vstack([embeddings[encoder]['Seen']['data'], embeddings[encoder]['Unseen']['data']])
        y_part = np.hstack([embeddings[encoder]['Seen'][rank], embeddings[encoder]['Unseen'][rank]])

        print(X_part.shape, y_part.shape)
            
        # ZSC-accuracy
        res_part = {}
        res_part["count"] = len(y_part)
        # Note that these evaluation metrics have all been converted to percentages
        res_part["AMI"] = 100.0 * zsc_pipeline(X_part, y_part)
        results[encoder][rank][partition_name] = res_part
        print(res_part["AMI"])
            

In [ ]:
print(results)

# Results
### The results of the analysis are shown below in two different formats
NOTE: The metric displayed in the graphs below is accuracy. Several other metrics have been calculated such as the balanced accuracy, all of which are stored in the 'results' dictionary.

In [ ]:
import json
with open(f"results_{dataset}_ZSC.json", "w") as f:
    json.dump(results, f)

In [ ]:
%matplotlib inline
import os
import pickle
import cProfile
import pstats
import warnings

import numpy as np
import sklearn
from sklearn.neighbors import KNeighborsClassifier, KDTree
from scipy.spatial import distance
from matplotlib import pyplot as plt
from obj_knn import FinBOL_GBOL
from tqdm import tqdm

warnings.filterwarnings("ignore")

dataset = "BIOSCAN-5M"
#data_folder = "BIOSCAN_5M_DNA_experiments/data"

import json

with open(f"results_{dataset}_ZSC.json", "r") as f:
    results = json.load(f)

rank_list = ["class", "order", "family", "genus", "species", "dna_bin"]
encoders = ["BarcodeBERT", "DNABERT-2", "DNABERT-S", "Hyena_DNA", "NT"]#, "DNABERT"]

x = np.arange(len(rank_list))  # the label locations
encoders = sorted(encoders,key=lambda x:results[x]['class']['test_seen + unseen']['AMI'],reverse=True)
width = 0.15  # the width of the bars
multiplier = -1

graph = {}
for encoder in encoders:
    graph[encoder] = [round(results[encoder][rank]['test_seen + unseen']['AMI'], 2) for rank in rank_list]

fig, ax = plt.subplots(layout='constrained')

for rank, measurement in graph.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=rank)
    multiplier += 1

for i in range(len(x)):
    for j, encoder in enumerate(encoders):
        plt.text(i+offset+(j-multiplier)*(width)+0.1, graph[encoder][i], graph[encoder][i], ha='center', rotation=60, fontsize='x-small')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Accuracy (%)')
ax.set_xlabel('Rank')

ax.set_title('BIOSCAN-5M ZSC encoder comparison')
ax.set_xticks(x + width, rank_list)
ax.legend(loc='upper right', ncols=2)
ax.set_ylim(0, 110)
plt.grid(axis="y")

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import json


plt.rcParams["font.family"] = "serif"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=["#C154A0", "#6495ED", "#FFBF00", "#922B21", "#1E8449", "#40E0D0", "#C18420"])


encoders = ["DNABERT-2", "DNABERT-S", "NT", "Hyena_DNA",  "BarcodeBERT"] #, "BarcodeBERT-5M"]

for encoder in encoders:
    plt.plot(rank_list, graph[encoder],"D-", label = encoder)
#plt.title('1NN-probing at different taxonomic levels')
plt.xlabel('Rank')
plt.ylabel('Accuracy (%)')
plt.grid(True)
plt.legend()
plt.savefig("BIOSCAN_5M_KNN_by_rank_cosine.eps", dpi=150)
plt.savefig("BIOSCAN_5M_KNN_by_rank_cosine.jpg", dpi=150)
plt.show()